In [39]:
import glob
import pandas as pd

In [47]:
dataset_name = "CAMI_Low"
prob = 50

In [48]:
def fasta_to_df(path):
    with open(path, 'r') as file:
        text = file.read()
    lines = [line for line in text.split('\n') if len(line) > 0]
    s = ''
    ids = []
    contigs = []
    for l in lines:
        if(l[0]=='>'):
            ids.append(l)        
            contigs.append(s)
            s = ''
        else:
            s += l    
    contigs.append(s)
    df = pd.DataFrame(
            {
                '#anonymous_gsa_id': ids,
                'contig': contigs[1:]
            })
    df['#anonymous_gsa_id'] = df['#anonymous_gsa_id'].apply(lambda x: x[1:])    
    return df

In [49]:
contigs = fasta_to_df(f"./Dataset/{dataset_name}/contigs.fna")
contigs.head()

,#anonymous_gsa_id,contig
0,RL|S1|C0,TCTCCGCCGACGAGAGCTACACGTTCGACCGTGATCACGCCGACTG...
1,RL|S1|C1,CCTGGCCATACTTTTCCTTTCATATAAGGTCCATCAGCACGTTTAC...
2,RL|S1|C2,TTCCATTGTAATTGCAACGAACAATCCCGTTGTTTCTGTATATGAC...
3,RL|S1|C3,CGTGGCCTCGGCAAGACCTTCACCCAGGGCACCTACTCGGTGCAGG...
4,RL|S1|C4,GAATATTCTACAACTATTACATCTTTTTGATAGCGTTGAACCAAAT...


# Convert list of fasta to tsv file

In [50]:
path = f'/home/jupyter/Final/Binning/{dataset_name}/Busybee/prob{prob}/'
seqId2newId = pd.read_csv(path + "contigs.tab",sep ='\t', header=None)
seqId2newId.columns = ['seqId','newId']
seqId2newId.head()

,seqId,newId
0,RL|S1|C0,0
1,RL|S1|C1,1
2,RL|S1|C2,2
3,RL|S1|C3,3
4,RL|S1|C4,4


In [51]:
newId2binId = pd.read_csv(path + "data/contigs.bin.table.tsv",sep ='\t', header=None)
newId2binId.columns = ['newId','binId']
newId2binId.head()

,newId,binId
0,315,0
1,543,1
2,2698,1
3,2801,0
4,3522,0


In [52]:
df = pd.merge(seqId2newId,newId2binId,how='inner',on='newId')[['seqId','binId']]
!rm -r {path}/*
df.to_csv(path + "contigs.tab", index=None, sep='\t',header=None)